# Representando texto - Capturando semántica

Generalmente cualquier tarea de PLN, comienza representando el texto en alguna forma numérica. En este capítulo se abordará diferentes caminos para hacerlo. 

En el presente capítulo se incluirán los siguiente tópicos:
* Poner documentos en una bolsa de palabras
* Construcción del modelo N-gram
* Representando textos con TF-IDF
* Uso de incrustaciones de palabras
* Entrenando su propio modelo de incrustaciones
* Representar frases - frasea2vec
* Uso de BERT en lugar de incrustaciones de palabras
* Comenzando con la búsqueda semántica

## Poner documentos en una bolsa de palabras

El método de la bolsa de palabras utiliza un texto de entrenamiento que le proporciona una lista de palabras que debe considerar. Al codificar nuevas oraciones, cuenta el número de ocurrencias que cada palabra hace en el documento, y el vector final incluye esos conteos para cada palabra en el vocabulario. Esta representación puede entonces ser alimentada en un algoritmo de aprendizaje automático.

In [6]:
import nltk

In [7]:
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

In [1]:
# Importamos las funciones a trabajar:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def read_text_file(filename):
    file = open(filename, "r", encoding="utf-8") 
    return file.read()

def preprocess_text(text):
    text = text.replace("\n", " ")
    return text

def divide_into_sentences_nltk(text):
    sentences = tokenizer.tokenize(text)
    return sentences

In [8]:
# Definir la función get_sentences, que leerá en el archivo de texto,
#preprocesar el texto, y dividirlo en oraciones:
def get_sentences(filename):
    text = read_text_file(filename)
    text = preprocess_text(text)
    sentences = divide_into_sentences_nltk(text)
    return sentences

In [9]:
# Crear una función que devolverá el vectorizador y la matriz final:
def create_vectorizer(sentences):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(sentences)
    return (vectorizer, X)

In [10]:
# Ahora, utilice las funciones antes mencionadas en el archivo pv.txt:
sentences = get_sentences("pv.txt")
(vectorizer, X) = create_vectorizer(sentences)

In [11]:
# Ahora vamos a imprimir la representación matricial del texto:
print(X)

  (0, 33)	1
  (0, 4)	1
  (0, 37)	1
  (0, 40)	1
  (0, 29)	1
  (0, 49)	1
  (0, 7)	1
  (0, 50)	1
  (0, 21)	2
  (0, 28)	1
  (0, 41)	1
  (0, 19)	1
  (0, 17)	1
  (0, 25)	1
  (0, 14)	1
  (0, 2)	1
  (0, 34)	1
  (0, 48)	1
  (0, 32)	1
  (1, 33)	1
  (1, 29)	1
  (1, 25)	1
  (1, 14)	1
  (1, 2)	1
  (1, 47)	3
  :	:
  (2, 46)	1
  (2, 15)	1
  (2, 12)	1
  (2, 13)	1
  (2, 26)	2
  (2, 5)	2
  (2, 55)	1
  (2, 0)	1
  (2, 53)	1
  (2, 1)	1
  (2, 23)	1
  (3, 2)	2
  (3, 34)	1
  (3, 26)	1
  (3, 45)	1
  (3, 52)	1
  (3, 43)	1
  (3, 56)	1
  (3, 35)	1
  (3, 27)	1
  (3, 11)	1
  (3, 3)	2
  (3, 39)	1
  (3, 38)	1
  (3, 20)	1


La matriz resultante es un objeto scipy.sparse.csr.csr_matrix. También se puede convertir en un objeto numpy.matrixlib.defmatrix.matrix, donde cada oración es un vector.

In [12]:
denseX = X.todense()

In [13]:
print(denseX)

[[0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 2 0 0 0 1 0 0 1 1 0 0 1 1 1 0
  0 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 2 0 1 0 0
  1 0 0 0 0 0 2 0 1 0 0 3 0 0 0 1 0 0 0 0 0]
 [1 1 0 0 0 2 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 2 0 0 1 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0]
 [0 0 2 2 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1
  0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1]]


In [14]:
# Podemos ver todas las palabras que se usaron en el conjunto de documentos:
print(vectorizer.get_feature_names())

['ac', 'an', 'and', 'are', 'array', 'as', 'basic', 'be', 'cells', 'changes', 'circuit', 'circuits', 'coupled', 'dc', 'developed', 'directly', 'effects', 'environment', 'equations', 'gui', 'here', 'in', 'including', 'inverter', 'irradiation', 'is', 'load', 'matching', 'matlab', 'model', 'new', 'of', 'paper', 'photovoltaic', 'presented', 'proper', 'pv', 'pva', 'results', 'simulated', 'simulation', 'simulink', 'solar', 'studies', 'temperature', 'test', 'tested', 'the', 'this', 'to', 'used', 'using', 'validation', 'via', 'was', 'well', 'with']


C:\Users\cdani\anaconda3\envs\NLP_cap3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
# La función transform espera una lista de documentos,
# por lo que crearemos una nueva lista con la oración como único elemento:
new_sentence = "I had seen little of Holmes lately."
new_sentence_vector = vectorizer.transform([new_sentence])


In [16]:
# Ahora podemos imprimir las matrices dispersas y densas de esta nueva frase:
print(new_sentence_vector)
print(new_sentence_vector.todense())

  (0, 31)	1
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


**Hay mas**

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
def read_text_file(filename):
    file = open(filename, "r", encoding="utf-8") 
    return file.read()

def preprocess_text(text):
    text = text.replace("\n", " ")
    return text

def divide_into_sentences_nltk(text):
    sentences = tokenizer.tokenize(text)
    return sentences

In [19]:
filename="sherlock_holmes_1.txt"
sherlock_holmes_text = read_text_file(filename)
sherlock_holmes_text = preprocess_text(sherlock_holmes_text)
sentences = divide_into_sentences_nltk(sherlock_holmes_text)

In [20]:
# Crear una nueva clase de vectorizador. Esta vez, utilice el argumento stop_words:
vectorizer = CountVectorizer(stop_words='english')

In [21]:
# Utilice el objeto vectorizador para obtener la matriz:
X = vectorizer.fit_transform(sentences)

In [23]:
# Podemos imprimir el vocabulario así:
print(vectorizer.get_feature_names())

['_the_', 'abhorrent', 'actions', 'adjusted', 'adler', 'admirable', 'admirably', 'admit', 'akin', 'balanced', 'cold', 'crack', 'delicate', 'distracting', 'disturbing', 'doubt', 'drawing', 'dubious', 'eclipses', 'emotion', 'emotions', 'excellent', 'eyes', 'factor', 'false', 'felt', 'finely', 'gibe', 'grit', 'heard', 'high', 'holmes', 'instrument', 'introduce', 'intrusions', 'irene', 'late', 'lenses', 'love', 'lover', 'machine', 'memory', 'men', 'mental', 'mention', 'mind', 'motives', 'nature', 'observer', 'observing', 'particularly', 'passions', 'perfect', 'placed', 'position', 'power', 'precise', 'predominates', 'questionable', 'reasoner', 'reasoning', 'results', 'save', 'seen', 'seldom', 'sensitive', 'sex', 'sherlock', 'sneer', 'softer', 'spoke', 'strong', 'temperament', 'things', 'throw', 'trained', 'veil', 'woman', 'world']


Ahora podemos aplicar el nuevo vectorizador a una de las oraciones del conjunto original y usar la función build_analyzer para ver más claramente el análisis de la oración:

In [24]:
new_sentence = "And yet there was but one woman to him, \
                and that woman was the late Irene Adler, of dubious and questionable memory."

In [26]:
new_sentence_vector = vectorizer.transform([new_sentence])
analyze = vectorizer.build_analyzer()
print(analyze(new_sentence))

['woman', 'woman', 'late', 'irene', 'adler', 'dubious', 'questionable', 'memory']


In [27]:
# Podemos imprimir el escaso vector de la frase así:
print(new_sentence_vector)

  (0, 4)	1
  (0, 17)	1
  (0, 35)	1
  (0, 36)	1
  (0, 41)	1
  (0, 58)	1
  (0, 77)	2
